# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [30]:
# Download nltk resources
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\q426259\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [101]:
# import libraries
import pandas as pd
import numpy as np
import pickle

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

In [32]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages_categories', engine) 

# Define features and target variables
X = df.message.values
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1).values

### 2. Write a tokenization function to process your text data

In [33]:
def tokenize(text):
    # Separate the sentences in words
    tokens = word_tokenize(text)
    
    # Remove stop words
    words = [w for w in tokens if w not in stopwords.words("english")]
    
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Lowercase, eliminate blank spaces and findin the root form of the words
    clean_tokens = []
    for tok in words:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [34]:
# Define pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [35]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# Train the classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [38]:
# Use the model to predict
y_pred = pipeline.predict(X_test)

In [62]:
# Get columns names
col_names = list(df.drop(['id', 'message', 'original', 'genre'], axis = 1).columns)

# Evaluate results
for col in np.arange(Y.shape[1]):
    labels = np.unique(y_pred[:,col])
    print("Column name: {}".format(col_names[col]))
    print(classification_report(y_test[:,col], y_pred[:,col], labels = labels))
    print("-----------------------------------------------------------------")

Column name: related
              precision    recall  f1-score   support

           0       0.76      0.26      0.39      1538
           1       0.81      0.97      0.88      4973
           2       0.67      0.23      0.34        43

    accuracy                           0.80      6554
   macro avg       0.74      0.49      0.54      6554
weighted avg       0.79      0.80      0.76      6554

-----------------------------------------------------------------
Column name: request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5442
           1       0.87      0.43      0.58      1112

    accuracy                           0.89      6554
   macro avg       0.88      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

-----------------------------------------------------------------
Column name: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00 

Column name: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6482

   micro avg       0.99      1.00      0.99      6482
   macro avg       0.99      1.00      0.99      6482
weighted avg       0.99      1.00      0.99      6482

-----------------------------------------------------------------
Column name: shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521

   micro avg       0.99      1.00      1.00      6521
   macro avg       0.99      1.00      1.00      6521
weighted avg       0.99      1.00      1.00      6521

-----------------------------------------------------------------
Column name: aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6476
           1       0.00      0.00      0.00        78

    accuracy                           0.99      6554
   macro avg       0.49      0.50    

In [96]:
# Mean f1_score
def model_performance(y_pred, y_test, col_names):
    scores = []
    for col in np.arange(len(col_names)):
        print("Column name: {}".format(col_names[col]))
        score = f1_score(y_test[:,col], y_pred[:,col], average='weighted')
        print(score)
        scores.append(score)
        print("-----------------------------------------------------------------")

    print("TOTAL F1-SCORE: {}".format(np.mean(scores)))

model_performance(y_pred, y_test, col_names)

Column name: related
0.7667915439324746
-----------------------------------------------------------------
Column name: request
0.8785505562571796
-----------------------------------------------------------------
Column name: offer
0.9922252380046092
-----------------------------------------------------------------
Column name: aid_related
0.7640963440096387
-----------------------------------------------------------------
Column name: medical_help
0.89307629581693
-----------------------------------------------------------------
Column name: medical_products
0.9290846368433641
-----------------------------------------------------------------
Column name: search_and_rescue
0.9659875282815387
-----------------------------------------------------------------
Column name: security
0.9770192682600755
-----------------------------------------------------------------
Column name: military
0.9530256492860938
-----------------------------------------------------------------
Column name: child_a

### 6. Improve your model
Use grid search to find better parameters. 

In [74]:
# Get parameters of the model
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001FBFA92EEE8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [83]:
# Set possible parameter values
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        #'clf__estimator__min_samples_leaf': [5, 10, 20],
        #'clf__estimator__max_depth': [10, 20, 50, 100, 200],
        'tfidf__smooth_idf': (True, False),
        #'tfidf__use_idf': (True, False)
}

# Do grid search
cv = GridSearchCV(pipeline, param_grid=parameters, return_train_score = True)

In [84]:
# Train the model
cv.fit(X_train, y_train)

# Predict
y_pred = cv.predict(X_test)

In [97]:
# Get model parameters
cv.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
               

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [98]:
# Get columns names
col_names = list(df.drop(['id', 'message', 'original', 'genre'], axis = 1).columns)

# Evaluate results
for col in np.arange(Y.shape[1]):
    labels = np.unique(y_pred[:,col])
    print("Column name: {}".format(col_names[col]))
    print(classification_report(y_test[:,col], y_pred[:,col], labels = labels))
    print("-----------------------------------------------------------------")

Column name: related
              precision    recall  f1-score   support

           0       0.76      0.28      0.40      1538
           1       0.81      0.97      0.88      4973
           2       0.67      0.28      0.39        43

    accuracy                           0.80      6554
   macro avg       0.74      0.51      0.56      6554
weighted avg       0.80      0.80      0.77      6554

-----------------------------------------------------------------
Column name: request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5442
           1       0.87      0.44      0.58      1112

    accuracy                           0.89      6554
   macro avg       0.88      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

-----------------------------------------------------------------
Column name: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00 

Column name: aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6476
           1       0.00      0.00      0.00        78

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

-----------------------------------------------------------------
Column name: other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6270
           1       0.00      0.00      0.00       284

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96      0.94      6554

-----------------------------------------------------------------
Column name: weather_related
              precision    recall  f1-score   support

           0       0.86      0.96      0.91      4758
           1     

In [103]:
# Estimate accuracy
print('Accuracy: {}'.format(np.mean(y_test == y_pred)))

Accuracy: 0.9456777879496829


In [104]:
# F1-score
model_performance(y_pred, y_test, col_names)

Column name: related
0.7667915439324746
-----------------------------------------------------------------
Column name: request
0.8785505562571796
-----------------------------------------------------------------
Column name: offer
0.9922252380046092
-----------------------------------------------------------------
Column name: aid_related
0.7640963440096387
-----------------------------------------------------------------
Column name: medical_help
0.89307629581693
-----------------------------------------------------------------
Column name: medical_products
0.9290846368433641
-----------------------------------------------------------------
Column name: search_and_rescue
0.9659875282815387
-----------------------------------------------------------------
Column name: security
0.9770192682600755
-----------------------------------------------------------------
Column name: military
0.9530256492860938
-----------------------------------------------------------------
Column name: child_a

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Adding stop words
# Adding pos='v' in the lemmatizer

### 9. Export your model as a pickle file

In [93]:
pickle.dump(cv, open('classifier_model_grid_search.pkl', 'wb'))

In [99]:
pickle.dump(pipeline, open('classifier_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.